In [1]:
def main():
    print("hello world")
    
if __name__ == "__main__":
    main()

hello world


In [1]:
import sklearn
import numpy as np
import csv
import scipy.io
import xlrd
import pandas as pd
from pandas import ExcelFile
from pandas import ExcelWriter
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [3]:
rootdir = 'C:/Users/Zman/Desktop/Jiook/code_embarc/data/connectome_fmri'

In [14]:
# method 1: find all patients in file, then compare ser_mor and fmri_list files, eliminate, and then compare remaining names with all patients, eliminate
# method 2: look at excel file ser_mor, get patients names that completed study, and search for these chars in the file system
# check if any string in list matches a segment of the selected list in file_list
def get_list():
    labels = []
    data = []
    real = []
    df = pd.read_excel('File.xlsx', sheetname='all_fmri_pat')
    col2 = df['label_w8_res_completer']
    inds = [(i for val in vals if (vals[i] = 1))]
    labels.append[df['label_remission'] for i in inds]
    col1 = df['RID']
    names = [col1[i] for i in col1 if (i in inds)]
    df = pd.read_excel('File.xlsx', sheetname='Book2')
    pat = df['Patients']
    real = [(pat[i] for i in pat if (1 for name in names if (name in pat[i])))]
    comp = [((rootdir + r + 'csv') for r in real)]
    for c in comp:
        with open(c, newline='') as csvfile:
            data.append(np.array(csv.reader(csvfile)))
    return data, labels # returns 86 patients with labels -- the patients in the non-placebo group who have completed the study



In [ ]:
def binary_classifier(features, labels): # a simple classifier that when given a bunch of correlation matrices and labels, will predict label on new matrices.
    train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42) # splits features w/ labels into training and testing sets
    gnb = GaussianNB()
    model = gnb.fit(train, train_labels)
    preds = model.pred(test, test_labels)
    print(accuracy_score(test_labels, preds))
    correct = []
    for (n : len(preds)):
        if (preds[n] == test_labels[n]):
            correct.append(1)
        else :
            correct.append(0)
    return correct, model

In [5]:
features, labels = get_list()
corr, mod = binary_classifier(features, labels)
m = binary_classifier(features, corr) # gives "sureness" of prediction
# problematic because this only has 33% of data. I want to record accuracy throughout process.

In [ ]:
def new_classifier(features, labels): # if sureness if lower than a threshold, then use classification of new
    threshold = 0.5
    train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42)
    svc = SVC()
    model = svc.fit(train, train_labels)
    for i in len(test):
        if (m.pred(test[i], test_labels[i]) < threshold):
            preds = svc.pred(test[i], test_labels[i])
        else:
            preds = m.pred(test[i], test_labels[i])
    print(accuracy_score(test_labels, preds))

In [ ]:
print(binary_classifier(features, labels))
print(new_classifier(features, labels))